# Todo
1. remove glove 
2. decrease embeddings size and use word2vec
3. use [center loss](https://medium.com/mlreview/experiments-with-a-new-loss-term-added-to-the-standard-cross-entropy-85b080c42446)

In [47]:
import warnings
warnings.filterwarnings("ignore")
import src.dataset as ds
import numpy as np
import keras.backend as K
from src.embeddings import extract_embedding_weights
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense
from keras.initializers import Constant
from keras import Sequential
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
import tensorflow as tf

In [48]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [49]:
X, y, tokenizer = ds.load_tokenized_data()
embedding_matrix = extract_embedding_weights()

100%|██████████████████████████████████████████████████████| 615/615 [00:01<00:00, 364.61it/s]


In [52]:
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity

In [53]:
def make_model():
    # load pre-trained word embeddings into an Embedding layer
    # note that we set trainable = False so as to keep the embeddings fixed
    EMBEDDING_DIM = 300
    num_words = len(tokenizer.word_index) + 1
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix),
                                input_length=1,
                                trainable=False)
    # define model
    model = Sequential()
    model.add(embedding_layer)
    model.add(CuDNNLSTM(50))
    model.add(Dense(num_words, activation='softmax'))
    model.summary()
    return model
    
    
def train(model, epochs=5):
    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[perplexity])
    # fit network
    model.fit(X, y, epochs=epochs, verbose=1, callbacks=[tensorboard])
    
    
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = np.array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
    		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [54]:
model = make_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1, 300)            2251800   
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 50)                70400     
_________________________________________________________________
dense_3 (Dense)              (None, 7506)              382806    
Total params: 2,705,006
Trainable params: 453,206
Non-trainable params: 2,251,800
_________________________________________________________________


In [55]:
train(model, epochs=10)

Epoch 1/10
193246/193246 [==============================] - 49s 252us/step - loss: 5.4143 - perplexity: 772.1470
Epoch 2/10
193246/193246 [==============================] - 46s 237us/step - loss: 4.7676 - perplexity: 430.7096
Epoch 3/10
193246/193246 [==============================] - 47s 242us/step - loss: 4.5263 - perplexity: 324.7318
Epoch 4/10
193246/193246 [==============================] - 47s 243us/step - loss: 4.3925 - perplexity: 281.3069
Epoch 5/10
193246/193246 [==============================] - 46s 237us/step - loss: 4.3051 - perplexity: 259.8325
Epoch 6/10
193246/193246 [==============================] - 47s 245us/step - loss: 4.2440 - perplexity: 248.9165
Epoch 7/10
193246/193246 [==============================] - 46s 240us/step - loss: 4.1988 - perplexity: 241.9717
Epoch 8/10
193246/193246 [==============================] - 47s 243us/step - loss: 4.1634 - perplexity: 237.6230
Epoch 9/10
193246/193246 [==============================] - 47s 243us/step - loss: 4.1353 - perp

In [37]:
generate_seq(model, tokenizer, 'hello', 10)

"hello . i 'm not the way . i 'm not"